In [1]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

#!pip install gmaps

!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
%matplotlib inline

# Import API key
from config import api_key
from config import g_key

gmaps.configure(api_key=g_key)

In [3]:
weather_data = pd.read_csv("cities.csv")
weather_data.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Iqaluit,75,CA,1605023904,78,63.75,-68.51,12.20,19.46
1,1,Ostrovnoy,97,RU,1605024404,83,68.05,39.51,34.83,20.02
2,2,Kununurra,1,AU,1605024122,78,-15.77,128.73,82.40,4.70
3,3,Laguna,1,US,1605024406,59,38.42,-121.42,37.99,0.76
4,4,Busselton,67,AU,1605024408,95,-33.65,115.33,50.00,6.06


In [4]:
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"]

In [5]:
fig = gmaps.figure(center=[0,0], zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
specific_city_df = weather_data.loc[(weather_data["Max Temp"] < 80) & (weather_data["Max Temp"] > 70) \
                                    & (weather_data["Wind Speed"] < 10) \
                                    & (weather_data["Cloudiness"] == 0)].dropna()

specific_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
147,147,Sakakah,0,SA,1605024106,29,29.97,40.21,77.00,4.70
153,153,Allāhganj,0,IN,1605024592,33,27.55,79.68,70.43,3.67
228,228,Bilma,0,NE,1605024683,21,18.69,12.92,76.75,3.65
229,229,Avarua,0,CK,1605024685,88,-21.21,-159.78,75.20,9.17
252,252,Sur,0,OM,1605024714,73,22.57,59.53,77.18,2.42
334,334,Tiznit Province,0,MA,1605024813,36,29.58,-9.50,73.69,7.00
417,417,Murud,0,IN,1605024915,61,18.33,72.97,74.53,4.81
460,460,Barra do Garças,0,BR,1605024969,74,-15.89,-52.26,78.80,6.51
493,493,Dābhol,0,IN,1605025010,62,17.60,73.17,71.47,5.17
531,531,Macas,0,EC,1605025057,57,-2.32,-78.12,75.20,2.24


In [7]:
hotel_df = specific_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
147,Sakakah,SA,29.97,40.21,
153,Allāhganj,IN,27.55,79.68,
228,Bilma,NE,18.69,12.92,
229,Avarua,CK,-21.21,-159.78,
252,Sur,OM,22.57,59.53,
334,Tiznit Province,MA,29.58,-9.50,
417,Murud,IN,18.33,72.97,
460,Barra do Garças,BR,-15.89,-52.26,
493,Dābhol,IN,17.60,73.17,
531,Macas,EC,-2.32,-78.12,


In [8]:
params = {
    "radius": 5000, 
    "types":"lodging", 
    "key":g_key}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_address = requests.get(base_url, params=params)
    
    hotel_address = hotel_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print ("Missing field/result... skipping.")
        
hotel_df

Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
147,Sakakah,SA,29.97,40.21,Fakhamat Al Orjoana Apartment
153,Allāhganj,IN,27.55,79.68,Rajesh TVS Motors
228,Bilma,NE,18.69,12.92,Expeditions Ténére voyages
229,Avarua,CK,-21.21,-159.78,Paradise Inn
252,Sur,OM,22.57,59.53,Sur Plaza Hotel
334,Tiznit Province,MA,29.58,-9.50,
417,Murud,IN,18.33,72.97,On Point Plaza
460,Barra do Garças,BR,-15.89,-52.26,Hotel Pousada Tropical
493,Dābhol,IN,17.60,73.17,Ashwamedh Resort
531,Macas,EC,-2.32,-78.12,Casa Upano - Guesthouse & Sanctuary


In [9]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))